<a href="https://colab.research.google.com/github/rahiakela/natural-language-processing-case-studies/blob/master/gpt-mechanism/text_generation_using_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Text generation using GPT-2

The first GPT model was introduced in a 2018 paper by Radford et al. from OpenAI – it demonstrated how a generative language model can acquire knowledge and process longrange dependencies thanks to pretraining on a large, diverse corpus of contiguous text. Two successor models (trained on more extensive corpora) were released in the following years: GPT-2 in 2019 (1.5 billion parameters) and GPT-3 in 2020 (175 billion parameters).

We will be making use of the excellent Transformers library created by Hugging Face(https://huggingface.co/). It abstracts away several components of the building process, allowing us to focus on the model performance and intended performance.

Reference:

https://huggingface.co/blog/how-to-generate
https://huggingface.co/transformers/model_doc/gpt2.html

## Setup

In [ ]:
# installing Transformers and TensorFlow 2.0 in one line
!pip install transformers[tf-cpu]

In [2]:
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

One of the advantages of the Transformers library – and a reason for its popularity, undoubtedly – is how easily we can download a specific model (and also define the appropriate tokenizer):

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-large")
GPT2 = TFGPT2LMHeadModel.from_pretrained("gpt2-large", pad_token_id=tokenizer.eos_token_id)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2-large.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


It is usually a good idea to fix the random seed to ensure the results are reproducible.

In [4]:
# settings

# for reproducability
SEED=34
tf.random.set_seed(SEED)

# maximum number of words in output text
MAX_LEN = 70

## Different Decoding Methods

Let us focus on the fact that how we decode is one of the most important decisions when using a GPT-2 model.

### Greedy search

With greedy search, the word with the highest probability is predicted as the next word in the sequence:

In [5]:
input_sequence = "I don't know about you, but there's only one thing I want to do after a long day of work"
input_sequence = "There are times when I am really tired of people, but I feel lonely too."

Once we have our input sequence, we encode it and then call a decode method:

In [6]:
# encode context the generation is conditioned on
input_ids = tokenizer.encode(input_sequence, return_tensors="tf")

# generate text until the output length (which includes the context length) reaches 70
greedy_output = GPT2.generate(input_ids, max_length=MAX_LEN)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
There are times when I am really tired of people, but I feel lonely too. I feel like I'm alone in the world. I feel like I'm alone in my own body. I feel like I'm alone in my own mind. I feel like I'm alone in my own heart. I feel like I'm alone in my own mind


As you can see, the results leave some room for improvement: the model starts repeating itself, because the high-probability words mask the less-likely ones so they cannot explore more diverse combinations.

### Beam search

A simple remedy is beam search: we keep track of the alternative variants, so that more comparisons are possible:

In [7]:
# set return_num_sequences > 1
beam_outputs = GPT2.generate(input_ids, max_length=MAX_LEN, num_beams=5, no_repeat_ngram_size=2, num_return_sequences=5, early_stopping=True)

print("")
print("Output:\n" + 100 * '-')

# now we have 5 output sequences
for i, beam_output in enumerate(beam_outputs):
  print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))


Output:
----------------------------------------------------------------------------------------------------
0: There are times when I am really tired of people, but I feel lonely too. I don't know what to do with myself."

"I feel like I can't do anything right now," she said. "I'm so tired."
1: There are times when I am really tired of people, but I feel lonely too. I don't know what to do with myself."

"I feel like I can't do anything right now," she says. "I'm so tired."
2: There are times when I am really tired of people, but I feel lonely too. I don't know what to do with myself."

"I feel like I can't do anything right now," she says. "I'm not sure what I'm supposed to be doing with my life."
3: There are times when I am really tired of people, but I feel lonely too. I don't know what to do with myself."

"I feel like I can't do anything right now," she says. "I'm not sure what I'm supposed to be doing."
4: There are times when I am really tired of people, but I feel lonely to

This is definitely more diverse – the message is the same, but at least the formulations look a little different from a style point of view.